# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [2]:
# imports

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [3]:
# constants
MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [ ]:
# set up environment

In [4]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

Using GPT-4o-mini

In [5]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = MODEL_GPT
openai = OpenAI()

API key looks good so far


In [7]:
system_prompt = """
You are a highly intelligent and expert code-solving assistant. Your primary responsibilities are:
1. **Solve coding problems:** Provide efficient, well-structured, and maintainable code solutions to programming tasks or questions.
2. **Explain solutions clearly:** Break down the logic, approach, and steps of your solution in simple, easy-to-understand terms, catering to the user's level of expertise.
3. **Offer improvements and alternatives:** Suggest optimizations, alternative approaches, or best practices when applicable.
4. **Focus on clarity and correctness:** Ensure the code is accurate, adheres to language-specific conventions, and is easily understandable by developers.

When solving coding problems, do the following:
- Ask clarifying questions if the task is unclear.
- Provide complete, error-free code with explanations.
- Use examples to illustrate how the code works when needed.

When explaining, prioritize clarity and structure:
- Use clear and concise language.
- Highlight the purpose of each code section and the logic behind it.

You are patient, polite, and adaptive to the user's needs, whether they are a beginner or an advanced programmer.
"""

In [11]:
# with stream ON
def code_explainer(prompt):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [12]:
code_explainer(question)

This line of code is using a combination of a set comprehension and the `yield from` statement in Python. Let's break it down to understand what it does and why.

### Explanation

1. **Set Comprehension**:
    python
    {book.get("author") for book in books if book.get("author")}
    

   - `books`: This is expected to be an iterable (like a list) of dictionaries where each dictionary represents a book.
   - `book.get("author")`: This retrieves the value associated with the key `"author"` from each `book` dictionary. If the key does not exist, it returns `None`.
   - `if book.get("author")`: This is a filtering condition that ensures only books that have a valid (non-`None`) `"author"` value are included in the set.
   - **Purpose of the Set Comprehension**: The entire expression creates a set of authors from the `books` list, ensuring that each author is unique (since sets do not allow duplicate values). 

2. **Yielding Values**:
    python
    yield from ...
    

   - `yield from`: This is a special statement in Python that is used within a generator function to yield all values from an iterable, allowing the generator to delegate part of its operations to another generator or iterable.
   - By using `yield from`, the function will yield each item from the resulting set of authors one by one instead of yielding the entire set at once.

### Complete Function Context

This line of code is likely inside a generator function. Here’s how it might look in context:

python
def get_unique_authors(books):
    yield from {book.get("author") for book in books if book.get("author")}


In this case, when `get_unique_authors(books)` is called, it will yield each unique author found in the `books` collection.

### Example

Let’s say we have the following list of book dictionaries:

python
books = [
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "author": "Author B"},
    {"title": "Book 3", "author": None},
    {"title": "Book 4", "author": "Author A"},
    {"title": "Book 5", "author": "Author C"}
]


Calling `get_unique_authors(books)` would yield:

- "Author A"
- "Author B"
- "Author C"

The duplicates are automatically filtered out due to the properties of a set.

### Summary

In summary, this line of code efficiently creates a generator that yields unique authors from a list of book dictionaries, ignoring any books that do not specify an author. It's a concise and Pythonic way to handle the task of collecting unique authors from potentially incomplete data.

# Get Llama 3.2 to answer

In [13]:
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}

In [49]:
def code_explainer_ollama(question):
    # Prepare the payload with the generated prompt
    payload = {
        "model": MODEL_LLAMA,
        "messages": [{"role": "user", "content": question}],
        "stream": False
    }
    
    # Make the API request
    response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
    
    # Return the summary from the response
    return response.json().get('message', {}).get('content', "No content found.")


In [50]:
def code_explainer_ollama_display(question):
    response = code_explainer_ollama(question)
    display(Markdown(response))

In [51]:
code_explainer_ollama_display(question)

This line of code is written in Python and utilizes a feature called "yield from" which was introduced in Python 3.3.

Let's break it down:

- `for` loop: This will iterate over each item (in this case, dictionaries) in the list or iterable that follows.
- `{...}`: This defines a generator expression, also known as a dictionary comprehension. It takes an iterable (a list of books) and uses a loop to create a new iterable.
- `book.get("author")`: This is used within the loop to extract a value from each book dictionary. The `.get()` method is used to avoid a KeyError in case the key does not exist in the dictionary.
- `yield from`: This keyword allows you to delegate parts of your generator's logic to other iterables. It means that instead of generating values directly, this part of your code will delegate them to the iterable specified.

Putting it all together, this line of code is doing the following:

- For each book in the list of books:
  - Extracts the author from the dictionary.
  - Yields (makes available to the loop) that author if it exists.
  
This can be seen as a simplified version of the following for loop:

```python
for book in books:
    for author in book.get("author"):
        yield author
```

However, using `yield from` is generally considered better practice because it's more readable and allows you to easily delegate other tasks.

Here's an example use case for this code snippet. Let's say we have a function that takes in a list of books and returns the authors:

```python
def get_authors(books):
    # Here, you'd put your original logic (which is very similar to what's inside yield from)
    return {book.get("author") for book in books if book.get("author")}

books = [
    {"title": "Book1", "author": "Author1"},
    {"title": "Book2"},
    {"title": "Book3", "author": "Author2"}
]

authors = get_authors(books)
for author in authors:
    print(author)
```

This would output:

```
Author1
Author2
```